<a href="https://colab.research.google.com/github/Dr-Carlos-Villasenor/VisionComputacional/blob/main/L07_RedesConvolucionales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visión Computacional
## Dr. Carlos Villaseñor
## Lección 7 - Redes Neuronales convolucionales

#### Cars196 Dataset
3D Object Representations for Fine-Grained Categorization
       Jonathan Krause, Michael Stark, Jia Deng, Li Fei-Fei
       4th IEEE Workshop on 3D Representation and Recognition, at ICCV 2013 (3dRR-13). Sydney, Australia. Dec. 8, 2013.


Primeramente importamos unos paquetes

In [1]:
# Importamos numpy y matplotlib
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import tensorflow as tf

# Importamos lo necesario para crear la red
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Rescaling, Resizing, Reshape
from keras.layers import Conv2D, MaxPooling2D
from keras import backend

# Importamos unos paquetes de sklearn para comprobar el modelo
from sklearn.metrics import classification_report, confusion_matrix

num_classes = 196
img_height = 40
img_width = 40

Cargamos el dataset de cars196

In [ ]:
(train_image, train_label) = tfds.load(
    'cars196',
    split=['train', 'test'],
    as_supervised=True,
    batch_size=-1
)

In [ ]:
def to_categorical(image, label):
  return image.reshape(image.shape[0]), tf.one_hot(label, num_classes)

ds_train = ds_train.map(
    to_categorical, num_parallel_calls=tf.data.AUTOTUNE)

ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

In [ ]:
ds_test = ds_test.map(
    to_categorical, num_parallel_calls=tf.data.AUTOTUNE)

ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

In [ ]:
if backend.image_data_format() == 'channels_first':
    input_shape = (3, img_height, img_width)
else:
    input_shape = (img_height, img_width, 3)

print(input_shape)

In [ ]:
def build_model():
  model = Sequential()

  model.add(Resizing(img_height, img_width))

  model.add(Rescaling(1./255, input_shape=input_shape))

  model.add(Conv2D(32,kernel_size=(3,3),
                   activation='relu',
                   input_shape=input_shape))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(0.25))

  model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(0.25))

  model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(0.25))

  model.add(Flatten())
  model.add(Dense(128, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(num_classes, activation='softmax'))

  model.compile(loss = keras.losses.categorical_crossentropy,
                optimizer = 'adam',
                metrics=['accuracy'])

  return model

In [ ]:
model = build_model()
model.summary()

In [ ]:
model.fit(ds_train,
          epochs=6,
          validation_data=ds_test)

In [ ]:
ypred=model.predict(xtest)
print('Salida de la red:\n', ypred[0])

In [ ]:
ypred = np.argmax(ypred, axis=1)
print('Predicciones:\n', ypred)

In [ ]:
print('Reporte de clasificación:\n', classification_report(ytest, ypred))
print('Matriz de confusión:\n', confusion_matrix(ytest, ypred))

In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

sample = np.random.randint(0,10000)
x = xtest[sample].reshape(28,28)
plt.imshow(x, cmap=plt.cm.gray)
plt.title('Prediction: ' + class_names[ypred[sample]])
plt.show()